# Rolling Returns Strategy

The principle of the strategy is pretty simple.

__Future Ret = Spot Ret + Roll Ret__

Then we can extract the rolling returns by going long the future and shorting the spot.

This works __as long as the sign of the roll returns don't change very quickly__

Basically this strategy is done under the principle that __rolling returns have a strong momentum and the slope of the futures curve tend to have same sign over long periods of time__

As sometimes we dont have access to the spot in commodities. We have to pass through an ETF, here, for oil we have gone through XLE, which tracks the energy companies. 



In [20]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import yfinance as yf

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

import sys
sys.path.append('/Users/educontreras/PycharmProjects/Quantitative-Finance/AlgoTrading_E_Chang')
import alpha_vantage
from Notebooks.utils import utils
from Notebooks.utils.config import alpha_vantage_api_key as av_key

## Step 1: Backwardation or contango?

We will compute the shape of the oil's future curve to see if it is in contango or in backwardation, if it is in backwardation we can expect to have positive rolling returns from the future contract. 

You want to be long future and short the spot to be exposed to rolling returns.

In [36]:
df = pd.read_csv(r'/Users/educontreras/PycharmProjects/Quantitative-Finance/AlgoTrading_E_Chang/data/inputDataDaily_CL_20120502.csv')
df['Date']=pd.to_datetime(df['Date'],  format='%Y%m%d').dt.date # remove HH:MM:SS
df.set_index('Date', inplace=True)

# Fitting gamma to forward curve
gamma=np.full(df.shape[0], np.nan)
for t in range(df.shape[0]):
    idx=np.where(np.isfinite(df.iloc[t, :]))[0]
    idxDiff=np.array(list(set(idx[1:])-set(idx)))
    if ((len(idx)>=5) & (all(idxDiff[0:4]==1))):
        FT=df.iloc[t, idx[:5]]
        T=sm.add_constant(range(FT.shape[0]))
        model=sm.OLS(np.log(FT.values), T)
        res=model.fit()
        #Gamma will contain the fitted slope of the futures curve
        gamma[t]=-12*res.params[1]
          
#Results: is the slope mean-reverting?
results=adfuller(gamma[np.where(np.isfinite(gamma))], maxlag=1, regression='c', autolag=None)
gamma=pd.DataFrame(gamma)
gamma.index = df.index
gamma.columns = ["Slope"]
gamma.fillna(method='ffill').dropna().iplot()

## Step 2: Import XLE and USO prices

When rolling returns are positive we want to be exposed to the first futures contract and short the spot to only be exposed to the rolling returns. 

In [53]:
prices = utils.get_prices(["USO", "XLE"], "2000-01-01")
ret = prices.pct_change()
prices = 1/2 * ret.iloc[:, 0] - 1/2 * ret.iloc[:, 1]
prices.name = "Roll"
overall = pd.merge(prices, gamma, left_index = True, right_index = True, how="inner")
overall["Slope"] = overall.Slope.apply(lambda x: 1 if x>=0 else -1)
overall = overall.dropna()

pos = overall.iloc[:, 0].shift(2)* overall.iloc[:, 1]
pos.name = "Daily Ret."
pos = pos.to_frame()
cumret = ((pos + 1).cumprod() - 1) * 100
cumret.iplot()

[*********************100%%**********************]  2 of 2 completed


## And the same with underlying not being a traded commodity?

We can try with the VIX. Bad thing: There is no ETF tracking the VIX by itself.

Retornos mas pobres debido a que no estamos con el underlying puro sino con ES1, y ademas es una curva que tiende a cambiar la slope sumamente rapido.